# __Fetch aoe2recs.com match details parallely__

### __Load libraries__

In [ ]:
from pymongo import MongoClient

from tqdm import tqdm

from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import os

from requests_ip_rotator import ApiGateway
import requests
import mechanize
import http.cookiejar
import random
import warnings

import multiprocessing
from joblib import Parallel, delayed

cores = 15 # API rate limiter forces to use 20
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### __Source functions__

In [ ]:
%run aoe2rec_api_modules.ipynb

### __Set up mongo connection__

In [ ]:
db_conn = setup_mongo_conn()

### __Delete invalid matches__

In [ ]:
delete_invalid_match_mongo(db_conn)

### __Fetch match details__

In [ ]:
# Choose between fetching new matches or missing ones
# Optionally delete bad records - id = -999

for index, row in fetch_proxies(https = True).iterrows():
    
    proxy = {'http' : 'https://' + row['IP Address'] + ':' + str(row['Port'])}
    
    print(proxy)
    
    try:
        fetch_matches_iterate_aoe2recs(db_conn, collect = cores, type = 'new', clean_empty = False, 
                                       cores = cores, proxy = proxy, progress_bar = False)
    except:
        print('Skipping to next IP !')
        pass